In [1]:
#install keras and tensorflow packages
#import the required packages
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.preprocessing.image import ImageDataGenerator

In [2]:
#creating the required model
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape = (64, 64, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Flatten())
model.add(Dense(units = 128, activation = 'relu'))
model.add(Dense(units = 1, activation = 'sigmoid'))

In [3]:
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [4]:
#training and validating the dataset
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
val_datagen = ImageDataGenerator(rescale = 1./255)
#training
training_set = train_datagen.flow_from_directory('C:/Users/musth/Desktop/shatru/Dataset/train',
                                                 target_size = (64, 64),
                                                 batch_size = 8,
                                                 class_mode = 'binary')
#validating
val_set = val_datagen.flow_from_directory('C:/Users/musth/Desktop/shatru/Dataset/val',
                                            target_size = (64, 64),
                                            batch_size = 8,
                                            class_mode = 'binary')

Found 888 images belonging to 2 classes.
Found 144 images belonging to 2 classes.


In [5]:
#training the model and obtaining accuracy
model.fit_generator(training_set,
                         steps_per_epoch = 10,
                         epochs = 50,
                         validation_data = val_set,
                         validation_steps = 2)

C:\Users\musth\anaconda3\lib\site-packages\keras\engine\training.py:1915: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/50
10/10 [==============================] - 53s 408ms/step - loss: 1.7787 - accuracy: 0.5014 - val_loss: 0.4919 - val_accuracy: 0.6875
Epoch 2/50
10/10 [==============================] - 2s 181ms/step - loss: 0.7664 - accuracy: 0.5657 - val_loss: 0.6869 - val_accuracy: 0.4375
Epoch 3/50
10/10 [==============================] - 2s 197ms/step - loss: 0.5661 - accuracy: 0.8013 - val_loss: 0.2860 - val_accuracy: 0.8750
Epoch 4/50
10/10 [==============================] - 2s 173ms/step - loss: 0.4456 - accuracy: 0.8273 - val_loss: 1.1834 - val_accuracy: 0.5000
Epoch 5/50
10/10 [==============================] - 1s 137ms/step - loss: 0.5996 - accuracy: 0.7220 - val_loss: 0.2244 - val_accuracy: 0.9375
Epoch 6/50
10/10 [==============================] - 2s 172ms/step - loss: 0.4497 - accuracy: 0.7751 - val_loss: 0.5559 - val_accuracy: 0.7500
Epoch 7/50
10/10 [==============================] - 2s 179ms/step - loss: 0.2837 - accuracy: 0.8454 - val_loss: 0.3064 - val_accuracy: 0.8750
Epoch

In [6]:
import tensorflow as tf

In [7]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk


In [8]:

from keras.models import model_from_json
import numpy as np
from keras.preprocessing import image

In [9]:
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
model.load_weights("model.h5")
print("Loaded model from disk")

Loaded model from disk


In [15]:
#function to give the output for each prediction
def classify(img_file):
    img_name = img_file
    test_image = image.load_img(img_name, target_size = (64, 64))

    test_image = image.img_to_array(test_image)
    test_image = np.expand_dims(test_image, axis=0)
    result = model.predict(test_image)

    if result[0][0] == 1:
        prediction = 'truck'
    else:
        prediction = 'ship'
    print(prediction,img_name)

In [18]:
#each file in the test dataset is checked
import os
path = 'C:/Users/musth/Desktop/shatru/Dataset/test'
files = []
# r=root, d=directories, f = files
for r, d, f in os.walk(path):
    for file in f:
     #can change the file type to jpg,png,jpeg as required
        if '.jpg' in file:
            files.append(os.path.join(r, file))

In [19]:
#printing prediction for each file
for f in files:
    classify(f)
    print('\n')

ship C:/Users/musth/Desktop/shatru/Dataset/test\23101504.jpg


truck C:/Users/musth/Desktop/shatru/Dataset/test\25295937.jpg


truck C:/Users/musth/Desktop/shatru/Dataset/test\27300711.jpg


truck C:/Users/musth/Desktop/shatru/Dataset/test\28723768.jpg


ship C:/Users/musth/Desktop/shatru/Dataset/test\299281.jpg


ship C:/Users/musth/Desktop/shatru/Dataset/test\299795.jpg


ship C:/Users/musth/Desktop/shatru/Dataset/test\299796.jpg


ship C:/Users/musth/Desktop/shatru/Dataset/test\299803.jpg


ship C:/Users/musth/Desktop/shatru/Dataset/test\554491.jpg


ship C:/Users/musth/Desktop/shatru/Dataset/test\560394.jpg


ship C:/Users/musth/Desktop/shatru/Dataset/test\563043.jpg


ship C:/Users/musth/Desktop/shatru/Dataset/test\565155.jpg


truck C:/Users/musth/Desktop/shatru/Dataset/test\62334922.jpg


truck C:/Users/musth/Desktop/shatru/Dataset/test\63292023.jpg


truck C:/Users/musth/Desktop/shatru/Dataset/test\64696696.jpg


truck C:/Users/musth/Desktop/shatru/Dataset/test\65561577.jpg




In [22]:
%%writefile app.py
import streamlit as st
from keras.models import model_from_json
#import tensorflow as tf

st.set_option('deprecation.showfileUploaderEncoding',False)
@st.cache(allow_output_mutation=True)
def load_model():
    json_file = open('model.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    model = model_from_json(loaded_model_json)
    model.load_weights("model.h5")
    return model

model=load_model()  
  

st.write("""
            SHIP or TRUCK Classifier
         """
        )
file=st.file_uploader("Please upload an image of TRUCK or SHIP",type=["jpg"])
#import cv2
from PIL import Image,ImageOps
import numpy as np
def import_and_predict(image_data,model):

    size=(64,64)
    image= ImageOps.fit(image_data,size,Image.ANTIALIAS)
    img=np.asarray(image)
    img_reshape=img[np.newaxis,...]
    prediction=model.predict(img_reshape)
    if prediction[0][0]==1:
        preds='TRUCK'
    else:
        preds='SHIIP'
    return preds  
  #return prediction


if file is None:
    st.text("Please upload an image file")
else:
    image=Image.open(file)
    st.image(image,use_column_width=True)
    predictions = import_and_predict(image,model)
    #class_names=['Mango','Jackfruit']
    string="This image is most likely:"+predictions
    st.success(string)

Overwriting app.py
